# SSB30806: Modelling in Systems Biology
## Python Tutorial pt.2: Simulating ODEs with python
This notebook will guide you through the steps needed to simulate systems of ordinary differential equations in Python.



### Step 1: Importing modules
First we import the modules we will need later on. We already introduced numpy and matplotlib in the python tutorial. For the actual numerical integration we will use the function **odeint** from the **scipy** module. Since this is the only function in this module that we will use, we will import it directly.

In [ ]:
#import necessary packages
import numpy as np #math and arrays
from scipy.integrate import odeint #differential equations
import matplotlib.pyplot as plt #plotting

In [ ]:
#run a "magic" command to make sure figures are rendered properly in the notebook
%matplotlib inline

### Step 2: Define a system of differential equations
Next, we need to define our system of differential equations. For this example, we will use a simple system with two equations:
\begin{align}
\frac{dx_1}{dt} &= a \cdot x_1 - b\cdot x_1\cdot x_2\\
\frac{dx_2}{dt} &= c \cdot x_1\cdot x_2 - d \cdot x_2
\end{align}
Here, $x_1$ and $x_2$ are the two state variables and $a$, $b$, $c$, and $d$ are parameters. This system can be thought of as a representation of simple predator prey dynamics, where $x_1$ is the density of the prey and $x_2$ is the density of the predator.

The numerical integration function we will use (**odeint**) requires us to define a function that calculates the derivatives of all state variables as specified by our differential equation. Note that we will not be calling this derivative function ourselves; it will be passed on to odeint which will call it internally. The numerical integration performed by odeint is essentially similar to the forward euler scheme explained in the lectures. It will start at an initial condition, calculate the derivatives there and use those to take a small step forward in time. This process is repeated until the specified end time is reached. The main difference with forward euler is that the size of the time steps is chosen automatically the steps are a bit more complicated and more accurate, but the process still relies on repeated evaluations of the derivatives. 

Therefore, the function for the derivatives of our ODE must therefore be defined in the way that the odeint function expects. 
It will pass a list of current values for all states to the first argument, the current time point to the second, and any parameters you gave it will be passed to the remaining arguments. This means that even though we will almost never use the value of the current time point, we still need to provide a parameter for it in our derivative function, because that is what odeint expects. The expected output is a list of derivatives of all states in the same order.

In [ ]:
def deriv(x, t, a, b, c, d):
    
    # Unpack the state variable information
    # The first argument contains the values of the state variables in order
    x1 = x[0] 
    x2 = x[1]

    # Calculate the derivatives of the two states
    dx1dt = a*x1 - b*x1*x2
    dx2dt = c*x1*x2 - d*x2
    
    # Make a list of all derivatives in the correct order
    dxdt = [dx1dt, dx2dt]

    # Return the derivatives
    return dxdt

### Step 3: Set the parameters for the simulation experiment
Now, we specify the precise conditions for which we want our ODEs simulated. First, we make a numpy array with all the time steps for which we want our states calculated. (Note that these are not the integration time steps used by odeint. If the integration requires smaller time steps to be accurate, odeint will use more steps internally but provide output only at the steps you specified.)

In [ ]:
# Define the simulation end time and number of time points for which you want output
t_end = 10
Nsteps = 200

# Make a numpy array with the time steps for which you want output
timepoints = np.linspace(0, t_end, Nsteps) 

Then, we can specify the values of the parameters we want to use. For now, let's set them all to 1. Parameters need to be provided in a tuple *in the same order* as they appear in the derivatives function we specified above. (Using the same variable names as in the derivatives function is recommended to prevent confusion, but python has no way of linking the variable names you use here to the ones from above, so it will look only at the order in which they are provided.)

In [ ]:
# Specify parameter values
a = 1
b = 1
c = 1
d = 1
parameters = (a,b,c,d)

The last thing we need to do before we can start the actual simulation is defining the initial conditions of the state variables. Again, these need to be provided in the same order that you used in the derivative function.

In [ ]:
# Set initial values for all state variables
x1_init = 2
x2_init = 4

x_init = [x1_init, x2_init] #combine into a list

### Step 4: Run the simulation experiment
Now we are ready to call the **odeint** function. This function takes three required arguments: the derivative function we defined at the beginning, the list of initial conditions, and the array of time points. If your derivative function needs any parameters, these should be passed in a tuple to an optional argument called **args**.

In [ ]:
# Calculate state variables at requested time points
x_t = odeint(deriv, x_init, timepoints, args = parameters)

The odeint function returns a numpy matrix with a column for each state variable and a row for each time point in the timepoints array, containing the values of these state variables at these time points.

In [ ]:
# Separate results into arrays for each state variable
x1_t = x_t[:,0]
x2_t = x_t[:,1]

### Step 5: Plot the results 
Finally, we can plot the results of the numerical integration using matplotlib.

In [ ]:
# Plot the state variables against time
plt.plot(timepoints, x1_t, label='x1')
plt.plot(timepoints, x2_t, label='x2')
plt.legend()
plt.xlabel('time')
plt.ylabel('states')
plt.show()

### Running multiple simulations
If you want to run multiple simulations, it may be convenient to wrap the entire simulation code into a function that takes the parameters and initial conditions as input arguments. You can have your function return both the array of timepoints and the simulation results by separating them with a comma.

In [ ]:
def simulateODE(a, b, c, d, x1_init, x2_init, t_end = 10, Nsteps = 200):

    # Make a numpy array with the time steps for which you want output
    timepoints = np.linspace(0, t_end, Nsteps)

    # Specify parameter values
    parameters = (a,b,c,d)

    # Set initial values for all state variables
    x_init = [x1_init, x2_init] #combine into a list

    # Calculate state variables at requested time points
    x_t = odeint(deriv, x_init, timepoints, args = parameters)
    
    # Return time points and state variables at those time points
    # This technically combines both variables into a tuple which can be unpacked after calling the function
    return timepoints, x_t


The same thing can be done for the plotting code.

In [ ]:
def plotODEoutput(timepoints, x_t):
    # Separate results into arrays for each state variable
    x1_t = x_t[:,0]
    x2_t = x_t[:,1]
    
    # Plot the state variables against time
    plt.plot(timepoints, x1_t, label='x1')
    plt.plot(timepoints, x2_t, label='x2')
    plt.legend()
    plt.xlabel('time')
    plt.ylabel('states')
    plt.show()
    

You can then call these functions multiple times for different settings without having to rewrite all that code over and over again. To split the output from the simulation function back into two separate arrays, simply provide two variable names separated by a comma.

In [ ]:
# Simulations for various parameter values and initial conditions
timepoints, x_t = simulateODE(1,1,1,1,12,10, t_end = 60)
plotODEoutput(timepoints, x_t)

timepoints, x_t = simulateODE(1,1,0.1,1,2,4, t_end = 30)
plotODEoutput(timepoints, x_t)

timepoints, x_t = simulateODE(1,1,0.1,1,12,10, t_end = 50)
plotODEoutput(timepoints, x_t)
